In [1]:
import numpy as np 
import pandas as pd
import torch 
import torch.nn as nn
import tensorflow as tf
import pandas as pd
from torch.utils.data import DataLoader , Dataset

In [2]:
# Import minist dataset
(x_train ,y_train ),(x_test  , y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000 , 784)
x_test = x_test.reshape(10000 , 784)
x_train.shape , y_train.shape , x_test.shape , y_test.shape , y_train.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


((60000, 784), (60000,), (10000, 784), (10000,), (60000,))

In [9]:
torch.cuda.is_available()

# make gpu as device
device = torch.device("cuda")
print(f"using device {device}")

using device cuda


In [10]:
# Converting into Tensors for Pytorch

x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [11]:
# Create optimize class
import torch.optim as opt

class NeuralNetwork(nn.Module):
    def __init__(self , feature_count):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(feature_count , 256 , bias= True),
            nn.ReLU(),
            nn.Linear(256 , 128 , bias= True),
            nn.ReLU(),
            nn.Linear(128 , 64 ),
            nn.ReLU(),
            nn.Linear(64 , 10),
        )
        
    def forward(self , features):
        out = self.network(features)
        return out

In [12]:
# Create Dataset class
class CustomeDataset(Dataset):
    def __init__(self, featues , lables):
        
        self.features = featues.to(dtype = torch.float32)
        self.features = self.features/255
        self.labels = lables.to(dtype = torch.long)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self , index):
        return self.features[index] , self.labels[index]

In [13]:
batch_size = 16
lr = 0.01
epochs = 20

In [14]:
dataset_train = CustomeDataset(x_train , y_train)
dataloder_train = DataLoader(dataset_train , shuffle= True , batch_size= batch_size)

# Model
model = NeuralNetwork(x_train.shape[1])
model = model.to(device)

loss_function = nn.CrossEntropyLoss()
opti = opt.Adam(model.parameters() , lr = lr)

In [15]:
x_train.shape

torch.Size([60000, 784])

In [ ]:
for epoch in range(epochs):
    total_epoch_loss = 0
    
    for train_features , train_lables in dataloder_train:

        train_features , train_lables = train_features.to(device) , train_lables.to(device)
        # forward pass
        y_pred = model(train_features)
        
        # find hte loss
        loss = loss_function(y_pred , train_lables)
        
        # reset gradient to zero
        opti.zero_grad()
         
        # backpropgation
        loss.backward()
        
        # paramter updates
        opti.step()
        
        total_epoch_loss += loss.item()
    
    print(f"Epoch : {epoch+1} , Loss : {total_epoch_loss/len(dataloder_train)}")

Epoch : 1 , Loss : 0.352196095780549
Epoch : 2 , Loss : 0.25598800998937804
Epoch : 3 , Loss : 0.21991362899654293
Epoch : 4 , Loss : 0.2001877747671429
Epoch : 5 , Loss : 0.17312541945756407
Epoch : 6 , Loss : 0.1877637961169802
Epoch : 7 , Loss : 0.15920141355254466
